### Dota Hero Grid Editor

Hero grids are stored in the following `.json` file in the steam folder:\
`C:\Program Files (x86)\Steam\userdata\YOUR_FRIEND_ID\570\remote\cfg\hero_grid_config.json`

Let's copy it into 
* our working directory - as **a dump file** to play with
* subfolder `./saved` of it^ to keep as **a backup file** in case we do something illegal

In [11]:
# pretty self-explanatory but dota_utils give functions
# to get hero names by their ids and vise versa
import json
import os
import shutil
import math

from config import DOTA_FRIEND_ID
import dota_utils

steam_cfg_loc = (
    f"C:\\Program Files (x86)\\Steam\\userdata\\{DOTA_FRIEND_ID}\\570\\remote\\cfg"
)
json_rel_loc = "\\hero_grid_config.json"  # rel- relative
src_steam_cfg = f"{steam_cfg_loc}{json_rel_loc}"  # source

out_temp = "./.temp"
if not os.path.isdir(out_temp):
    os.mkdir(out_temp)

dst_backup = "./backup.json"  # destination for backup
shutil.copy2(src_steam_cfg, out_temp)
_ = shutil.copy2(
    src_steam_cfg, f"{out_temp}{dst_backup}"
)  # _ = just so it doesn't print into output

### READ WRITE CELLS

In [12]:
# READ FROM DUMP FILE
with open(".temp/hero_grid_config.json") as json_file:
    data = json.load(json_file)


def write_and_copy():
    # WRITE INTO DUMP FILE
    with open(".temp/hero_grid_config.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    # COPY DUMP FILE BACK TO STEAM_CFG_LOC FOLDER WHEN WE ARE DONE
    src_dump = out_temp + json_rel_loc
    shutil.copy2(src_dump, steam_cfg_loc)

### Info to plan the Hero Grid
`data` - is the dictionary that is gonna be dumped into the json file back. 

```python
MAX_X = 1200
MAX_Y = 598
```
remember that this is not "pixels" - it just means that hero grid is overall 1200x598 in its own Dota 2 coordinates for the json file.

`data['configs'][0]` is my main Draft grid\
`data['configs'][1]` is Dota Plus levels grid

In [13]:
from typing import NamedTuple

MAX_X, MAX_Y = 1200, 592
# MAX_Y = 599 before CROWN FALL UPDATE


class P(NamedTuple):
    x: int | float
    y: int | float
    w: int | float
    h: int | float


class Grid:
    def __init__(self, config_index: int, categories: dict[str, P]) -> None:
        self.config_index: int = config_index
        self.categories: dict[str, P] = categories

    def update_categories(self):
        for name, position in self.categories.items():
            for category in data["configs"][self.config_index]["categories"]:
                if category["category_name"] == name:
                    category["x_position"] = position.x
                    category["y_position"] = position.y
                    category["width"] = position.w
                    category["height"] = position.h
                    break
            else:
                raise KeyError(
                    f'Category with name "{name}" does not exist in this hero grid.'
                    "Please add this category into your actual grid in Dota 2 client or in file yourself."
                )

### Dota Plus Levels Grid

In [14]:
# changeable constants
left_limit_x = 450  # the line between my left and right grid parts
right_part_w = MAX_X - left_limit_x
height = 100
delta = 12

bronze_delta = 10
bronze5_h = 100

# Categories

dota_plus_grid_categories = {  # match these names with the ones you have in the hero grid
    "Grandmaster": P(
        0,
        0,
        left_limit_x / 2,
        height,
    ),
    "Master": P(left_limit_x / 2, 0, left_limit_x / 2, height),
    "Platinum": P(0, height + delta, left_limit_x, height),
    "Gold": P(0, 2 * (height + delta), left_limit_x, height),
    "Silver": P(0, 3 * (height + delta), left_limit_x, MAX_Y - 3 * (height + delta)),
    "Bronze 5, 475<=Xp": P(left_limit_x, 0, right_part_w / 3, bronze5_h),
    "Bronze 5, 300<=Xp<475": P(
        left_limit_x + right_part_w / 3, 0, right_part_w / 3, bronze5_h
    ),
    "Bronze 5, XP<300": P(
        left_limit_x + right_part_w * 2 / 3, 0, right_part_w / 3, bronze5_h
    ),
    "Bronze 4-": P(
        left_limit_x,
        bronze_delta + bronze5_h,
        right_part_w,
        MAX_Y - bronze_delta - bronze5_h,
    ),
}


class DotaPlusGrid(Grid):
    def __init__(self) -> None:
        super().__init__(config_index=1, categories=dota_plus_grid_categories)

    def sort_by_dota_plus_xp(self):
        # todo: hmm, idk how to do it considering
        # I have data private
        # there is a way from Stratz to get it though
        # it's not really accurate
        pass


dota_plus_grid = DotaPlusGrid()
dota_plus_grid.update_categories()
write_and_copy()

### My default picking screen Grid

In [ ]:
left_limit_x = 500

# POSITIONS
height = 95
rows = 5  # 5 positions in dota
ban_space = 12

# SIXTH ROW DETAILS
sixth_row_h = 69
sixth_row_w_ratio = 3 / 9
# if it's not a natural number then it might bug out with 1 pixel lines
turbo_bans_w = math.ceil(sixth_row_w_ratio * left_limit_x)

delta_pos = (MAX_Y - height * rows - sixth_row_h - ban_space) / (rows - 1)

default_roles_grid_categories = (
    {
        f"pos{i + 1}": P(x=0, y=(height + delta_pos) * i, w=left_limit_x, h=height)
        for i in range(rows)
    }
    | {"Turbo bans": P(x=0, y=MAX_Y - sixth_row_h, w=turbo_bans_w, h=sixth_row_h)}
    | {
        "Grind/Arcana/Style/D+/Cavern": P(
            x=turbo_bans_w,
            y=MAX_Y - sixth_row_h,
            w=left_limit_x - turbo_bans_w,
            h=sixth_row_h,
        )
    }
)


class DefaultRolesGrid(Grid):
    def __init__(self) -> None:
        super().__init__(config_index=0, categories=default_roles_grid_categories)

    def fix_attribute_categories(self):
        indexes = {"str": 5, "agi": 6, "int": 7, "all": -1}

        all_hero_ids: list[int] = []

        for count, (primary_attribute, idx) in enumerate(indexes.items()):
            category = data["configs"][self.config_index]["categories"][idx]

            category["x_position"] = left_limit_x
            category["y_position"] = count * MAX_Y / 4
            category["width"] = MAX_X - left_limit_x
            category["height"] = MAX_Y / 4

            # Fix alphabet if needed.
            category["hero_ids"] = sorted(
                category["hero_ids"], key=lambda x: dota_utils.heroes[x].name.casefold()
            )

            # Print attribute warning mismatch.
            for hero_id in category["hero_ids"]:
                hero = dota_utils.heroes[hero_id]
                if primary_attribute != hero.primary_attribute:
                    print(f"Warning! Primary Attribute mismatch for {hero!r}.")

            # Check if all heroes are present and not duplicated.
            all_hero_ids += category["hero_ids"]

        # Check for duplicates within attributes half of the screen.
        seen: set[int] = set()
        dupes = {x for x in all_hero_ids if x in seen or seen.add(x)}
        if dupes:
            print(f"Warning! Duplicates found! {dupes}")

        # Check for missing heroes within attributes half of the screen.
        missing_ids = set(dota_utils.heroes.keys()) - set(all_hero_ids)
        if missing_ids:
            print(
                f"Warning! Missing heroes are found! {[dota_utils.heroes[hero_id] for hero_id in missing_ids]}"
            )


default_grid = DefaultRolesGrid()
default_grid.update_categories()
default_grid.fix_attribute_categories()
write_and_copy()

[73, 2, 96, 81, 51, 135, 69, 49, 107, 7, 103, 59, 23, 104, 54, 77, 129, 60, 84, 57, 110, 137, 14, 28, 71, 18, 29, 98, 19, 83, 100, 108, 85, 42, 1, 4, 62, 61, 56, 6, 106, 41, 72, 123, 8, 80, 48, 94, 82, 9, 114, 10, 89, 44, 12, 15, 32, 11, 93, 35, 46, 109, 95, 70, 20, 47, 63, 68, 66, 5, 55, 119, 87, 58, 121, 74, 64, 90, 52, 31, 25, 26, 138, 36, 111, 76, 13, 45, 39, 131, 86, 79, 27, 75, 101, 17, 34, 37, 112, 30, 22, 102, 113, 3, 65, 38, 78, 50, 43, 33, 91, 97, 136, 53, 88, 120, 16, 128, 67, 105, 40, 92, 126, 21]
Warning! Missing heroes are found! [<Hero self.id=145 self.name='Kez'>, <Hero self.id=99 self.name='Bristleback'>]
